## 7.3 오토인코더 클래스

In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets

In [2]:
class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch # 학습 횟수
        self.learning_rate = learning_rate # 최적화기의 하이퍼파라미터
        
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim]) # 입력 레이어 데이터셋 정의
        
        with tf.name_scope('encode'): # 이름 범위 아래에서 가중치와 편향을 정의함으로써 디코더의 가중치와 편향으로부터 구분할 수 있습니다
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'): # 디코더의 가중치와 편향은 이 이름 범위 하에서 정의됩니다
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
            
        self.x = x # 메소드 변수들입니다
        self.encoded = encoded
        self.decoded = decoded
        
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded)))) # 재현 비용을 정의합니다
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss) # 최적화기를 선택합니다
        self.saver = tf.train.Saver() # 학습을 진행하면서 모델 파라미터들을 저장하기 위해 saver를 설정합니다
        
    def train(self, data):
        num_samples = len(data)
        with tf.Session() as sess: # 텐서플로우 세션을 시작하고 모든 변수들을 초기화합니다
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch): # 생성자에 정의된 수만큼 반복합니다
                for j in range(num_samples): # 한번에 하나씩 데이터 항목을 이용해 신경망을 학습시킵니다
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: [data[j]]})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l)) # 10번마다 출력해 줍니다
                    self.saver.save(sess, './model.ckpt') # 학습된 파라미터를 파일로 저장합니다
            self.saver.save(sess, './model.ckpt')
            
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt') # 학습된 파라미터를 불러옵니다
            hidden, reconstructed = sess.run([self.encoded, self.decoded],feed_dict={self.x: data}) # 입력을 복원해 줍니다
        print('input', data)
        print('compressed', hidden)
        print('reconstructed', reconstructed)
        return reconstructed

## 7.6 Autoencoder 클래스 사용하기

In [3]:
hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

epoch 0: loss = 4.762742519378662
epoch 10: loss = 1.944270372390747
epoch 20: loss = 0.8767220973968506
epoch 30: loss = 0.4848218262195587
epoch 40: loss = 0.41396087408065796
epoch 50: loss = 0.4091811776161194
epoch 60: loss = 0.4066024720668793
epoch 70: loss = 0.4022473096847534
epoch 80: loss = 0.395643413066864
epoch 90: loss = 0.3860918879508972
epoch 100: loss = 0.3730546534061432
epoch 110: loss = 0.35694658756256104
epoch 120: loss = 0.33847108483314514
epoch 130: loss = 0.31794899702072144
epoch 140: loss = 0.2952507436275482
epoch 150: loss = 0.27193784713745117
epoch 160: loss = 0.2523600459098816
epoch 170: loss = 0.24104058742523193
epoch 180: loss = 0.23876768350601196
epoch 190: loss = 0.24279622733592987
epoch 200: loss = 0.2499697059392929
epoch 210: loss = 0.2581237554550171
epoch 220: loss = 0.2660641372203827
epoch 230: loss = 0.27346184849739075
epoch 240: loss = 0.2804086208343506
INFO:tensorflow:Restoring parameters from ./model.ckpt
input [[8, 4, 6, 2]]
comp

array([[6.401917 , 2.8301592, 5.366955 , 1.8683255]], dtype=float32)

## 7.7 배치 도우미 함수

In [4]:
def get_batch(X, size):
    a = np.random.choice(len(X), size, replace=False)
    return X[a]

## 7.8 배치 학습

In [5]:
tf.reset_default_graph()

In [6]:
class Autoencoder_batch:
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch # 학습 횟수
        self.learning_rate = learning_rate # 최적화기의 하이퍼파라미터
        
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim]) # 입력 레이어 데이터셋 정의
        
        with tf.name_scope('encode'): # 이름 범위 아래에서 가중치와 편향을 정의함으로써 디코더의 가중치와 편향으로부터 구분할 수 있습니다
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'): # 디코더의 가중치와 편향은 이 이름 범위 하에서 정의됩니다
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
            
        self.x = x # 메소드 변수들입니다
        self.encoded = encoded
        self.decoded = decoded
        
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded)))) # 재현 비용을 정의합니다
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss) # 최적화기를 선택합니다
        self.saver = tf.train.Saver() # 학습을 진행하면서 모델 파라미터들을 저장하기 위해 saver를 설정합니다
        
    def train(self, data, batch_size=10):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(500): # 배치만큼 반복하게 됩니다
                    batch_data = get_batch(data, batch_size) # 랜덤하게 선택된 배치에 대해 최적화기를 실행합니다
                    l, _ = sess.run([self.loss, self.train_op],feed_dict={self.x: batch_data})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
            
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt') # 학습된 파라미터를 불러옵니다
            hidden, reconstructed = sess.run([self.encoded, self.decoded],feed_dict={self.x: data}) # 입력을 복원해 줍니다
        print('input', data)
        print('compressed', hidden)
        print('reconstructed', reconstructed)
        return reconstructed

In [7]:
hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder_batch(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

epoch 0: loss = 2.9653894901275635
epoch 10: loss = 0.6367222666740417
epoch 20: loss = 0.4280121922492981
epoch 30: loss = 0.4102773666381836
epoch 40: loss = 0.34543317556381226
epoch 50: loss = 0.3505767285823822
epoch 60: loss = 0.230178564786911
epoch 70: loss = 0.3078746795654297
epoch 80: loss = 0.23413164913654327
epoch 90: loss = 0.250286340713501
epoch 100: loss = 0.3602640628814697
epoch 110: loss = 0.3141750991344452
epoch 120: loss = 0.30150073766708374
epoch 130: loss = 0.20863670110702515
epoch 140: loss = 0.30766263604164124
epoch 150: loss = 0.25836998224258423
epoch 160: loss = 0.3910287022590637
epoch 170: loss = 0.24819539487361908
epoch 180: loss = 0.2692684531211853
epoch 190: loss = 0.2519698441028595
epoch 200: loss = 0.3069063723087311
epoch 210: loss = 0.29161790013313293
epoch 220: loss = 0.2231282740831375
epoch 230: loss = 0.23868906497955322
epoch 240: loss = 0.4563460350036621
INFO:tensorflow:Restoring parameters from ./model.ckpt
input [[8, 4, 6, 2]]
com

array([[6.8641934, 2.8006907, 6.1956196, 2.2268982]], dtype=float32)

## 7.10 추출된 CIFAR-10 데이터셋으로부터 읽어오기

In [8]:
tf.reset_default_graph()

In [9]:
from scipy.misc import imread, imresize
import pickle

def unpickle(file): # CIFAR-10 파일을 읽어와서 로드된 사전을 반환합니다
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

## 7.11 모든 CIFAR-10 파일을 읽어 메모리에 저장하기

In [10]:
names = unpickle('./cifar-10-batches-py/batches.meta')['label_names']
data, labels = [], []
for i in range(1, 6): # 6개의 파일에 대해 반복합니다
    filename = './cifar-10-batches-py/data_batch_' + str(i)
    batch_data = unpickle(filename) # 파일을 로드하여 파이썬 사전을 획득합니다
    if len(data) > 0:
        data = np.vstack((data, batch_data['data'])) # 데이터 샘플의 행은 각각의 샘플을 표현하기 때문에 수직으로 쌓게 됩니다
        labels = np.hstack((labels, batch_data['labels'])) # 레이블은 1차원이기 때문에 수평으로 쌓게 됩니다
    else:
        data = batch_data['data']
        labels = batch_data['labels']

## 7.12 CIFAR-10 이미지를 흑백으로 변환하기

In [11]:
def grayscale(a):
    return a.reshape(a.shape[0], 3, 32, 32).mean(1).reshape(a.shape[0], -1)

In [12]:
data = grayscale(data)

In [13]:
x = np.matrix(data)
y = np.array(labels)

horse_indices = np.where(y == 7)[0]
horse_x = x[horse_indices]

print(np.shape(horse_x))  # (5000, 3072)

input_dim = np.shape(horse_x)[1]
hidden_dim = 100

ae = Autoencoder_batch(input_dim, hidden_dim)
ae.train(horse_x)

(5000, 1024)
epoch 0: loss = 89.40890502929688
epoch 10: loss = 59.64217758178711
epoch 20: loss = 47.695709228515625
epoch 30: loss = 42.410377502441406
epoch 40: loss = 48.668739318847656
epoch 50: loss = 56.84235382080078
epoch 60: loss = 45.0015983581543
epoch 70: loss = 44.886409759521484
epoch 80: loss = 50.54345703125
epoch 90: loss = 49.140350341796875
epoch 100: loss = 51.03417205810547
epoch 110: loss = 50.56010818481445
epoch 120: loss = 57.118656158447266
epoch 130: loss = 47.41543197631836
epoch 140: loss = 54.57421875
epoch 150: loss = 46.37538528442383
epoch 160: loss = 51.31253433227539
epoch 170: loss = 47.668418884277344
epoch 180: loss = 49.60804748535156
epoch 190: loss = 43.20268249511719
epoch 200: loss = 42.5274772644043
epoch 210: loss = 48.62804412841797
epoch 220: loss = 49.7543830871582
epoch 230: loss = 50.54779815673828
epoch 240: loss = 42.87990951538086
